In [1]:
import torch
torch.cuda.is_available()

False

In [2]:
torch.device("cuda")

device(type='cuda')

In [4]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from collections import deque, namedtuple
import copy
from swig import swig
from tqdm import tqdm

In [ ]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

In [3]:
class Brain(nn.Module):
    def __init__(self, input_shape, output_size):
        super(Brain, self).__init__()
        self.conv1 = nn.Conv2d(input_shape[2], 16, kernel_size=4, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=2)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, stride=1)

        conv_out_size = self._get_conv_output(input_shape)

        self.fc1 = nn.Linear(conv_out_size, 256)
        self.fc2 = nn.Linear(256, output_size)

    def _get_conv_output(self, shape):
        input = torch.rand(1, *shape)
        input = input.permute(0, 3, 1, 2)
        output = self.conv3(self.conv2(self.conv1(input)))
        return int(np.prod(output.size()))

    def forward(self, x):
        if x.dim() == 3:
            x = x.unsqueeze(0)
        if x.size(1) != 3:
            x = x.permute(0, 3, 1, 2)
        
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [7]:
import gymnasium as gym
env = gym.make("CarRacing-v2")
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
action_size = env.action_space.shape[0]
print("State shape: ", state_shape)
print("state size: ", state_size)
print("action size: ", action_size)

DependencyNotInstalled: Box2D is not installed, you can install it by run `pip install swig` followed by `pip install "gymnasium[box2d]"`

In [47]:
learning_rate = 1e-3
minibatch_size = 64 
discount_factor = 0.99
replay_buffer_size = int(1e5)
tau = 1e-3 

In [48]:
class ReplayMemory:
    def __init__(self, capacity, state_shape, action_dim, device):
        self.capacity = capacity
        self.device = device
        self.memory = deque(maxlen=capacity)
        self.state_shape = state_shape
        self.action_dim = action_dim

    def push(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        batch = random.sample(self.memory, batch_size)
        
        # Pre-allocate numpy arrays
        states = np.zeros((batch_size, *self.state_shape), dtype=np.float32)
        actions = np.zeros((batch_size, self.action_dim), dtype=np.float32)
        rewards = np.zeros((batch_size, 1), dtype=np.float32)
        next_states = np.zeros((batch_size, *self.state_shape), dtype=np.float32)
        dones = np.zeros((batch_size, 1), dtype=np.float32)

        # Fill the numpy arrays
        for i, (state, action, reward, next_state, done) in enumerate(batch):
            states[i] = state
            actions[i] = action
            rewards[i] = reward
            next_states[i] = next_state
            dones[i] = done

        # Convert numpy arrays to tensors
        return (
            torch.from_numpy(states).pin_memory().to(self.device,non_blocking=True),
            torch.from_numpy(actions).to(self.device),
            torch.from_numpy(rewards).to(self.device),
            torch.from_numpy(next_states).to(self.device),
            torch.from_numpy(dones).to(self.device)
        )

    def __len__(self):
        return len(self.memory)

In [57]:
class OUNoise:
    def __init__(self, size, mu=0., theta=0.15, sigma=0.2):
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.reset()

    def reset(self):
        self.state = copy.copy(self.mu)

    def sample(self):
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.random.randn(len(x))
        self.state = x + dx
        return self.state.tolist()  # Return as a list

In [71]:
class Agent:
    def __init__(self, state_shape, action_size, learning_rate=1e-4, replay_buffer_size=10000, minibatch_size=32, tau=1e-3):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.state_shape = state_shape
        self.action_size = action_size
        
        self.actor = Brain(state_shape, action_size).to(self.device)
        self.critic = Brain(state_shape, 1).to(self.device)
        self.target_actor = Brain(state_shape, action_size).to(self.device)
        self.target_critic = Brain(state_shape, 1).to(self.device)
        
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=learning_rate)
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=learning_rate)
        
        self.memory = ReplayMemory(replay_buffer_size, state_shape, action_size, self.device)
        self.minibatch_size = minibatch_size
        self.tau = tau

    def act(self, state):
        self.actor.eval()
        with torch.no_grad():
            action = self.actor(state).cpu().numpy().squeeze()
        self.actor.train()
        return np.clip(action, -1, 1)

    def step(self, state, action, reward, next_state, done):
        self.memory.push(state, action, reward, next_state, done)
        
        if len(self.memory) > self.minibatch_size:
            experiences = self.memory.sample(self.minibatch_size)
            self.learn(experiences, 0.99)

    def learn(self, experiences, gamma):
        states, actions, rewards, next_states, dones = experiences

        # Update critic
        with torch.no_grad():
            next_actions = self.target_actor(next_states)
            Q_targets_next = self.target_critic(next_states)
            Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))
        
        Q_expected = self.critic(states)
        critic_loss = F.mse_loss(Q_expected, Q_targets)
        
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        # Update actor
        actor_loss = -self.critic(states).mean()
        
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        # Update target networks
        self.soft_update(self.critic, self.target_critic, self.tau)
        self.soft_update(self.actor, self.target_actor, self.tau)

    def soft_update(self, local_model, target_model, tau):
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

In [72]:
agent = Agent(state_shape, action_size)

In [83]:
def train_agent(agent, env, number_episodes=2000, max_timesteps_per_episode=1000, target_score=200.0):
    scores_window = deque(maxlen=100)
    epsilon = 1.0
    epsilon_decay = 0.995
    epsilon_min = 0.01

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    def normalize_state(state):
        return (state - state.mean()) / (state.std() + 1e-8)

    for episode in tqdm(range(1, number_episodes + 1), desc="Training Progress"):
        state, _ = env.reset()
        state = normalize_state(state)
        score = 0
        
        for t in range(max_timesteps_per_episode):
            if np.random.random() < epsilon:
                action = env.action_space.sample()
            else:
                state_tensor = torch.from_numpy(state).float().unsqueeze(0).to(device)
                with torch.no_grad():
                    action = agent.act(state_tensor)
            
            next_state, reward, done, _, _ = env.step(action)
            next_state = normalize_state(next_state)
            
            agent.step(state, action, reward, next_state, done)
            
            state = next_state
            score += reward
            
            if done:
                break
        
        scores_window.append(score)
        epsilon = max(epsilon_min, epsilon * epsilon_decay)
        
        if episode % 100 == 0:
            avg_score = np.mean(scores_window)
            print(f'\rEpisode {episode}\tAverage Score: {avg_score:.2f}\tEpsilon: {epsilon:.2f}')
            torch.save(agent.actor.state_dict(), f'checkpoint_actor_{episode}.pth')
            torch.save(agent.critic.state_dict(), f'checkpoint_critic_{episode}.pth')
        
        if np.mean(scores_window) >= target_score:
            print(f'\nEnvironment solved in {episode-100:d} episodes!\tAverage Score: {np.mean(scores_window):.2f}')
            torch.save(agent.actor.state_dict(), 'checkpoint_actor_final.pth')
            torch.save(agent.critic.state_dict(), 'checkpoint_critic_final.pth')
            return True
    
    print(f'\nEnvironment not solved. Final Average Score: {np.mean(scores_window):.2f}')
    torch.save(agent.actor.state_dict(), 'checkpoint_actor_final.pth')
    torch.save(agent.critic.state_dict(), 'checkpoint_critic_final.pth')
    return False

train_agent(agent, env)

KeyboardInterrupt: 

In [80]:
def normalize_state(state):
    return (state - state.mean()) / (state.std() + 1e-8).astype(np.float32)

number_episodes = 5000  # Increased from 2000
max_timesteps_per_episode = 1000
scores_window = deque(maxlen=100)

epsilon = 1.0
epsilon_decay = 0.995
epsilon_min = 0.01

for episode in range(1, number_episodes + 1):
    state, _ = env.reset()
    state = normalize_state(state).astype(np.float32)
    agent.noise.reset()
    score = 0
    
    for t in range(max_timesteps_per_episode):
        if np.random.random() < epsilon:
            action = env.action_space
            
        else:
            action = agent.act(state)
        if isinstance(action, list):
            action = np.array(action, dtype=np.float32)
            
        next_state, reward, done, _, _ = env.step(action)
           # Normalize and convert types
        next_state = next_state.astype(np.float32) if isinstance(next_state, np.ndarray) else next_state
        
        reward = np.float32(reward)
        agent.step(state, action, reward, next_state, done)
        state = next_state
        score += reward
        if done:
            break
    
    scores_window.append(score)
    epsilon = max(epsilon_min, epsilon * epsilon_decay)
    
    # print(f'\rEpisode {episode}\tAverage Score: {np.mean(scores_window):.2f}\tEpsilon: {epsilon:.2f}')
    if episode % 100 == 0:
        print(f'\rEpisode {episode}\tAverage Score: {np.mean(scores_window):.2f}\tEpsilon: {epsilon:.2f}')
        # Save checkpoints periodically
        torch.save(agent.actor.state_dict(), f'checkpoint_actor_{episode}.pth')
        torch.save(agent.critic.state_dict(), f'checkpoint_critic_{episode}.pth')
    
    if np.mean(scores_window) >= 200.0:
        print(f'\nEnvironment solved in {episode-100:d} episodes!\tAverage Score: {np.mean(scores_window):.2f}')
        torch.save(agent.actor.state_dict(), 'checkpoint_actor_final.pth')
        torch.save(agent.critic.state_dict(), 'checkpoint_critic_final.pth')
        break

# If not solved, save the final model anyway
if np.mean(scores_window) < 200.0:
    print(f'\nEnvironment not solved. Final Average Score: {np.mean(scores_window):.2f}')
    torch.save(agent.actor.state_dict(), 'checkpoint_actor_final.pth')
    torch.save(agent.critic.state_dict(), 'checkpoint_critic_final.pth')

TypeError: in method 'b2RevoluteJoint___SetMotorSpeed', argument 2 of type 'float32'

In [ ]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gymnasium.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v2')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()